# Part 4: Alternative max-marginal decoding

### Data Preprocessing 

In [1]:
import numpy as np
import pandas as pd
import time

def createDf(language,file):
    '''
    language = 'CN' , 'EN', 'FR', 'SG'
    '''
    tweets = []          # A list of all the tweets (Each tweet is a list)
    word_count, tweet_count = 0, 0
    # Import training data
    with open('./' + language + file, encoding='utf8') as f:
        training_lines = f.readlines()

        # For each line in the file
        for line in training_lines: 

            # If line is empty (i.e. we enter a new tweet)
            if line in['\n', '\r\n']: # Initialize a new tweet, reset word count
                if word_count != 0: #If the previous tweet was not empty, increase tweet count
                    tweet_count += 1
                word_count = 0

            else:
                # Remove the spaces in each line
                stripped = line.strip().split(" ")
                if len(stripped) == 2:
                    if word_count == 0:
                        tweets.append([tweet_count, word_count,'None','Start'])
                        word_count += 1
                    tweets.append([tweet_count, word_count] + stripped)
                    word_count += 1
                    
    df = pd.DataFrame(tweets,columns=['Tweet', 'Word', 'Observation', 'State'])
    df = df.set_index(['Tweet', 'Word'])
    return df

In [2]:
def createDfDevin(language,file):
    '''
    language = 'CN' , 'EN', 'FR', 'SG'
    '''
    tweets = []          # A list of all the tweets (Each tweet is a list)
    word_count, tweet_count = 0, 0
    # Import training data
    with open('./' + language + file, encoding='utf8') as f:
        training_lines = f.readlines()

        # For each line in the file
        for line in training_lines: 

            # If line is empty (i.e. we enter a new tweet)
            if line in['\n', '\r\n']: # Initialize a new tweet, reset word count
                if word_count != 0: #If the previous tweet was not empty, increase tweet count
                    tweet_count += 1
                word_count = 0

            else:
                # Remove the spaces in each line
                stripped = line.strip().split(" ")
                if len(stripped) == 1:
                    if word_count == 0:
                        tweets.append([tweet_count, word_count,'None'])
                        word_count += 1
                    tweets.append([tweet_count, word_count] + stripped)
                    word_count += 1
                    
    df = pd.DataFrame(tweets,columns=['Tweet', 'Word', 'Observation'])
    df = df.set_index(['Tweet', 'Word'])
    
    return df

In [3]:
def getTweet(df, tweetNumber):
    """
    Inputs:
    df: dataframe of all tweets
    tweetNumber: which tweet to access and extract

    Output:
    obs_list: list of observations for a specified tweet 
    """
    df_resetindex = df.reset_index()
    tweet_df = df_resetindex.loc[df_resetindex['Tweet'] == tweetNumber]
    
    # Convert tweet dataframe to a list
    tweet_list = tweet_df.values.T.tolist()
    
    # Append a None at the end of observation to account for 'Stop' state
    obs_list = tweet_list[2]
    obs_list.append('None')
    
    # returns a list of observations
    return obs_list

In [4]:
def Count_State(df):
    '''
    Get Count(i) and Count(j)
    '''
    states_counter = df.groupby('State').count()
    return states_counter

### Obtain Emission Probabilities

In [5]:
def Count_Emission(df):
    df = df.copy()
    df["Count"] = 1
    
    emission_count = df.groupby(['State','Observation'],).count().reset_index(level = 'Observation')
    emission_count = emission_count.join(Count_State(df), rsuffix = '_State')
    emission_count = emission_count.drop('Observation_State',axis=1)    
    emission_count["emission"] = emission_count['Count'] / emission_count['Count_State']
    emission_count = emission_count.drop('Count_State',axis=1)
    
    return emission_count

def Replace_With_Unk(df, k):
    emission_count = df.copy()
    drop_table = emission_count.groupby(['Observation'],).sum()
    drop_table = drop_table.loc[drop_table['Count'] < k].reset_index()
    emission_count['Observation'].loc[emission_count['Observation'].isin(drop_table['Observation'])] = '#UNK#'
    emission_count = emission_count.groupby(['State','Observation'],).sum()
    
    return emission_count

### (5 pts) Write a function that estimates the transition parameters from the training set using MLE (maximum likelihood estimation):

#### Please make sure the following special cases are also considered: q(STOP|yn) and q(y1|START).

<img src="images/mle2.jpg">

In [6]:
def Count_Transistion(df):
    transistion = df.copy()
    transistion['J'] = transistion['State']
    transistion['J'] = transistion['J'].shift(-1)
    transistion['J'].loc[transistion['J'] == 'Start'] = 'Stop'
    transistion['J'].loc[pd.isnull(transistion['J'])] = 'Stop'
    count_transistion = transistion.groupby(['State','J']).count()
    
    # Create Full table of transistion permutations
    states = Count_State(df).reset_index().as_matrix()[:-1,0]
    length = states.shape[0] + 1
    start = np.reshape(np.concatenate((['Start'],states)),(1,-1))
    Stop = np.reshape(np.concatenate((states,['Stop'])),(1,-1))
    states = np.vstack((np.repeat(start,length),np.ravel(np.repeat(Stop,length,axis=0)))).T
    states = pd.DataFrame(states, columns=['State','J'])
    states['Observation'] = 0
    states = states.set_index(['State','J'])
    count_transistion = states.join(count_transistion, how= 'left', lsuffix= '2').drop('Observation2', axis = 1).fillna(0)
    
    # Compute transistion probabilities
    count_transistion = count_transistion.join(Count_State(df), lsuffix='_trans', rsuffix='_state')
    count_transistion['aij'] = count_transistion['Observation_trans'] / count_transistion['Observation_state']
    print("Transistion Dataframe created.")
    
    return count_transistion

In [7]:
def ObtainEmission(state, observation,count_emission):
    df = count_emission.reset_index()
    renamed = df.loc[(df['State']==state) & (df['Observation']==observation)].emission.as_matrix()
    if len(renamed) == 0:
        return 0
    else:
        return renamed[0]

In [8]:
def alpha(tweet,j,count_state,count_emission,count_transistion,order,all_words,alpha_list,n_states):
    n = len(tweet)
    
    # Base case
    if j == 1:
        a1 = [count_transistion.get(key) for key in [('Start',state) for state in order]]
        a1 = np.asarray(a1)
        alpha_list.append(a1)
        return alpha(tweet,2,count_state,count_emission,count_transistion,order,all_words,alpha_list,n_states)
    
    # Final case
    elif j == n-1:
        return (alpha_list)
    
    # Recursive case
    else:
        a = []
        
        # Replace unknown words with #UNK#
        if tweet[j-1] not in all_words:
            tweet[j-1] = '#UNK#'
            
        for i in range(n_states):
            a_transition = [count_transistion.get(key) for key in [(state,order[i]) for state in order]]
            a_transition = np.asarray(a_transition)
            a_emission = [0 if i is None else i for i in [count_emission.get(key)\
                                         for key in [(state,tweet[j-1]) for state in order]]]
            a_emission = np.asarray(a_emission)
            previous_a = alpha_list[len(alpha_list)-1]

            summation = np.multiply(previous_a,a_transition)
            summation = np.multiply(a_emission,summation)

            a.append(np.sum(summation))
            
        alpha_list.append(a)
        
        return alpha(tweet,j+1,count_state,count_emission,count_transistion,order,all_words,alpha_list,n_states)

In [9]:
def beta(tweet,k,count_state,count_emission,count_transistion,order,all_words,beta_list,n_states):
    n = len(tweet)
    
    # Base case
    if k == -1 :
        k = n-1
        b_transition = [count_transistion.get(key) for key in [(state,'Stop') for state in order]]
        
        if tweet[k-1] not in all_words:
            tweet[k-1] = '#UNK#'
        b_emission = [0 if i is None else i for i in [count_emission.get(key)\
                                         for key in [(state,tweet[k-1]) for state in order]]]
        summation = np.multiply(b_transition,b_emission)
        beta_list.append(summation)
        
        return beta(tweet,k-1,count_state,count_emission,count_transistion,order,all_words,beta_list,n_states)
    
    # Final case
    elif k == 1:
        return beta_list
    
    # Recursive case
    else:
        if tweet[k-1] not in all_words:
            tweet[k-1]='#UNK#'
        b = []
        
        for l in range(n_states):
            b_transition = [count_transistion.get(key) for key in [(order[l],state) for state in order]]
            b_emission = [0 if i is None else i for i in [count_emission.get(key)\
                                         for key in [(order[l],tweet[k-1])]]]
            previous_b=beta_list[len(beta_list)-1]
            summation = np.multiply(previous_b,b_transition)
            b.append(np.sum(b_emission*summation))
            
        beta_list.append(b)
        
        return beta(tweet,k-1,count_state,count_emission,count_transistion,order,all_words,beta_list,n_states)

In [10]:
# Obtain the optimal tag
def y_max(tweet,count_state,count_emission,count_transistion,order,all_words,n_states):
    y_max_list = []
    alpha_list = []
    beta_list = []
    a_list = alpha(tweet,1,count_state,count_emission,count_transistion,order,all_words,alpha_list,n_states)
    b_list = beta(tweet,-1,count_state,count_emission,count_transistion,order,all_words,beta_list,n_states)
    b_list_reversed = b_list[::-1]
    
    for i in range(0,len(a_list)):
        summation = np.multiply(a_list[i],b_list_reversed[i])
        tag = summation.argmax()
        y_max_list.append(order[tag])
        
    return y_max_list

In [11]:
def Predict(df, df_test):
    # Create dataframes
    count_state = Count_State(df)
    count_emission = Replace_With_Unk(Count_Emission(df), 3)
    all_words = count_emission.reset_index().Observation.values   
    count_transistion = Count_Transistion(df)
    count_transistion_swap = count_transistion.swaplevel(i = 'State', j = 'J').sort_index()
    order = Count_State(df).reset_index().as_matrix()[:-1,0]
    n_states = len(order)

    # Convert to dictionary
    count_state = count_state.to_dict()['Observation']
    count_emission = count_emission.to_dict()['emission']
    count_transistion = count_transistion.to_dict()['aij']
    count_transistion_swap = count_transistion_swap.to_dict()['aij']
    
    print('Predicting now...')
    df_test_size = df_test.reset_index().Tweet.max()+1
    predictions = []
    s_time = time.time()
    
    for x in range(df_test_size):
        if x % 50 == 0:
            print('x:',x,'time:',time.time()-s_time)
        tweet = getTweet(df_test,x)
        observations = y_max(getTweet(df_test,x),count_state,count_emission,count_transistion,order,all_words,n_states)
        predictions = predictions + ["{} {}\n".format(a_, b_) for a_, b_ in zip(tweet[1:-1],observations)] + ['\n']
        
    return predictions

In [12]:
def __main__(lang): 
    np.seterr(divide='ignore')#Ignore log zero warnings
    df = createDf(lang,'/train')
    df_test = createDfDevin(lang,'/dev.in')
    
    predictions = Predict(df, df_test)
    with open('%s/dev.p4.out'%lang, 'w',encoding='utf8') as f:
        f.write(''.join(predictions))
        print("Saved.")
        
'''
Languages: 'CN' , 'EN', 'FR', 'SG'
Change the language below accordingly
'''
language = 'CN'
__main__(language)

C:\Users\Pinar\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Transistion Dataframe created.
Predicting now...
x: 0 time: 0.0
x: 50 time: 1.1265294551849365
x: 100 time: 2.3797101974487305
x: 150 time: 3.526888847351074
x: 200 time: 4.835927724838257
Saved.
